In [1]:
import pandas as pd 
import tensorflow
from tensorflow import keras
from keras import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

df = pd.read_csv("../Churn_Modelling.csv")

In [2]:
df

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [3]:
df = df.drop(["RowNumber", "CustomerId", "Surname"], axis=1)

In [4]:
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [5]:
le = LabelEncoder()
for col in df.select_dtypes("object"):
    df[col] = le.fit_transform(df[col])
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,0,0,42,2,0.00,1,1,1,101348.88,1
1,608,2,0,41,1,83807.86,1,0,1,112542.58,0
2,502,0,0,42,8,159660.80,3,1,0,113931.57,1
3,699,0,0,39,1,0.00,2,0,0,93826.63,0
4,850,2,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,0,1,39,5,0.00,2,1,0,96270.64,0
9996,516,0,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,0,0,36,7,0.00,1,0,1,42085.58,1
9998,772,1,1,42,3,75075.31,2,1,0,92888.52,1


In [6]:
Y = df["Exited"]
X = df.drop(["Exited"], axis=1)
Xtrain, Xtest,ytrain,ytest = train_test_split(X,Y, test_size=0.2, shuffle=True, random_state=42)

In [7]:
#scaling es muy importante para gardient descent en NN, SVM, distance measurementes, knn 
scaler = MinMaxScaler()
#important to apply it after splitting bc it affects data leakage and overfitting if combined with xtest , por eso debe ser independiente
Xtrain = scaler.fit_transform(Xtrain)
#se aplican los parameters learned from the training set
Xtest = scaler.transform(Xtest)


In [8]:
import tensorflow as tf
from keras.layers import Dense, Dropout, BatchNormalization
from keras.optimizers import Adam
from sklearn.metrics import classification_report,confusion_matrix
import numpy as np


unique_classes = np.unique(ytrain)



In [9]:
from keras.callbacks import EarlyStopping


early_stopping = EarlyStopping(
    monitor='recall',    # Metric to monitor
    patience=10,            # Number of epochs with no improvement after which training will be stopped
    restore_best_weights=True,  # Restore model weights from the epoch with the best value of the monitored quantity
    mode = "max"
    
)

model = Sequential()
model.add(Dense(units=400,  activation="relu", kernel_initializer = "he_normal" ))
model.add(BatchNormalization())
model.add(Dropout(0.4))
model.add(Dense(units=600, activation="relu", kernel_initializer = "he_normal"  ))
model.add(BatchNormalization())
model.add(Dropout(0.4))
model.add(Dense(units=300, activation="relu", kernel_initializer = "he_normal" ))
model.add(BatchNormalization())
model.add(Dense(units=100, activation="relu", kernel_initializer = "he_normal" ))
model.add(BatchNormalization())
model.add(Dense(units=100, activation="relu", kernel_initializer = "he_normal" ))
model.add(BatchNormalization())
model.add(Dense(units=1, activation="sigmoid"))

model.compile(optimizer=Adam(), loss="binary_crossentropy", metrics=["recall"])
model.fit(x=Xtrain, y=ytrain, epochs=100, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - loss: 0.6512 - recall: 0.4256 - val_loss: 0.4403 - val_recall: 0.0870
Epoch 2/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.4549 - recall: 0.2987 - val_loss: 0.4020 - val_recall: 0.3634
Epoch 3/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.4191 - recall: 0.3270 - val_loss: 0.3792 - val_recall: 0.4006
Epoch 4/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.3820 - recall: 0.3724 - val_loss: 0.3748 - val_recall: 0.3571
Epoch 5/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.3906 - recall: 0.3655 - val_loss: 0.3816 - val_recall: 0.4783
Epoch 6/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.3802 - recall: 0.4066 - val_loss: 0.3622 - val_recall: 0.3727
Epoch 7/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.3832 - recall: 0.4044 - val_loss: 0.3634 - val_recall: 0.4658
Epoch 8/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.3615 - recall: 0.4084 - val_loss: 0.3716 - val_recall: 0.4130


In [10]:
loss, recall = model.evaluate(Xtest, ytest)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.3981 - recall: 0.5368 


In [11]:
ypred = (model.predict(Xtest) > 0.5).astype("int32")

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


In [12]:
ypred

array([[0],
       [0],
       [0],
       ...,
       [1],
       [0],
       [0]])

In [13]:
print(classification_report(ytest, ypred))

              precision    recall  f1-score   support

           0       0.89      0.94      0.91      1607
           1       0.66      0.51      0.58       393

    accuracy                           0.85      2000
   macro avg       0.78      0.72      0.74      2000
weighted avg       0.84      0.85      0.85      2000



In [14]:
import pickle
with open("neural3.pkl", "wb") as file:
    pickle.dump(model, file)